In [1]:
from tqdm import tqdm
import os
import torch
import json
import numpy as np
import pickle
import pandas as pd
from collections import defaultdict
from itertools import accumulate
from omegaconf import OmegaConf
from run_utils import get_etab, optimize_sequence, string_to_int, process_configs, cat_neighbors_nodes, rewrite_pdb_sequences, chain_to_partition_map, inter_partition_contact_mask
from potts_mpnn_utils import PottsMPNN, tied_featurize, nlcpl, parse_PDB, parse_PDB_seq_only, loss_nll
import etab_utils as etab_utils

In [12]:
# Define the configuration dictionary
# REPLACE paths and values with your specific requirements
config_dict = {
    "dev": "cuda" if torch.cuda.is_available() else "cpu",
    "out_dir": "outputs/example_sequence_outputs",
    "out_name": "sample_run",
    "input_list": "inputs/example_list_sample_seqs.txt",  # Path to a text file containing list of PDBs and optionally chain information (e.g., 1abc designs all chains for pdb 1abc, 1abc|A:B|C designs chains A and B with visible chain C, 1abc|A:B:C| designs chains A, B, and C
    "input_dir": "inputs/example_pdbs",  # Directory containing .pdb files
    "chain_dict_json": None, # Dictionary with lists of designed and visible chains for each pdb (only needed if specific chains need to be designed and info is not in input_list
    "model": { # Model configs (change check_path as needed, don't change others unless using a retrained model)
        "check_path": "vanilla_model_weights/pottsmpnn_msa_20.pt",
        "hidden_dim": 128,
        "edge_features": 128,
        "potts_dim": 400,
        "num_layers": 3,
        "num_edges": 48,
    },
    "inference": {
        "num_samples": 1, # Number of sequences to sample for each structure (must be 1 if optimizing)
        "temperature": 0.1, # Sampling temperature (default 0.1)
        "noise": 0.0, # Noise to add during inference (default 0.0)
        "skip_gaps": False, # Whether to skip gaps in structure (default False, but for some downstream applications like forward folding it can be convenient to set to True)
        "fix_decoding_order": True, # Whether to use a fixed decoding order for each structure
        "decoding_order_offset": 0, # Offset if you want a fixed decoding order different from the standard fixed decoding order
        "optimization_mode": "potts", # Optimization protocol (default "potts"); for no optimization set to "none"; for node optimization set to "nodes"
        "optimization_temperature": 0.0, # Optimization temperature (default 0.1)
        "binding_energy_optimization": "none", # Indicates how to incorporate binding energies into optimization (default "none"); for joint binding and stability optimization use "both"; for only binding use "only"
        "binding_energy_json": None, # "inputs/example_binding_energy_partitions_seqs.json", # Path to json with information about how chains should be separated for binding energy calculation for optimization (required for binding energy optimization)
        "binding_energy_cutoff": 8, # Angstrom cutoff for which residues to optimize with binding energies (binding energies only accurate for residues close to the interface)
        "optimize_pdb": False, # Indicates that sequences in .pdb files should be optimized.
        "optimize_fasta": "", # Indicates that sequences in the input .fasta file should be optimized (must have headers matching pdb_list)
        "write_pdb": True, # Indicates that .pdb files with the best sequence for each structure should be created
        "fixed_positions_json": "", # Filename for a json containing fixed positions (1 indexed list of positions in the structure) for each chain in each structure
        "pssm_json": "", # Filename for a json containing fixed positions (1 indexed list of positions in the structure) for each chain in each structure
        "omit_AA_json": "", # Filename for a json containing pssm info (coefficients of chain length and bias of shape chain length by vocab (21 for standard model, 22 for MSA model)) for each chain in each structure
        "bias_AA_json": "", # Filename for a json with residue bias info (vocab)
        "tied_positions_json": "", # Filename for a json containing tied position information for each structure
        "bias_by_res_json": "", # Filename for a json containing residue bias info (shape chain length by vocab) for each chain in each structure
        "omit_AAs": [], # List of residues to omit from prediction
        "pssm_threshold": 0.0, # A value between -inf + inf to restrict per position AAs
        "pssm_multi": 0.0, # A value between [0.0, 1.0], 0.0 means do not use pssm, 1.0 ignore MPNN predictions
        "pssm_log_odds_flag": False, # 0 for False, 1 for True
        "pssm_bias_flag": False # 0 for False, 1 for True
    }
}
config_dict["model"]["vocab"] = 22 if 'msa' in config_dict["model"]["check_path"] else 21
if config_dict["inference"]["temperature"] == 0: config_dict["inference"]["temperature"] = 1e-6
if config_dict["inference"]["optimization_temperature"] == 0: config_dict["inference"]["optimization_temperature"] = 1e-6
if config_dict["inference"]["optimize_pdb"] or config_dict["inference"]["optimize_fasta"]: config_dict["inference"]["num_samples"] = 1
if config_dict["inference"]["optimization_mode"] == "none": config_dict["inference"]["optimization_mode"] = ""

# Convert dictionary to an OmegaConf object for dot-notation access (cfg.model.vocab)
cfg = OmegaConf.create(config_dict)
dev = cfg.dev
print(f"Configuration loaded. Running on device: {dev}")

Configuration loaded. Running on device: cuda


In [13]:
print("Loading model checkpoint...")
checkpoint = torch.load(cfg.model.check_path, map_location='cpu', weights_only=False) 

model = PottsMPNN(
    ca_only=False, 
    num_letters=cfg.model.vocab, 
    vocab=cfg.model.vocab, 
    node_features=cfg.model.hidden_dim, 
    edge_features=cfg.model.hidden_dim, 
    hidden_dim=cfg.model.hidden_dim, 
    potts_dim=cfg.model.potts_dim, 
    num_encoder_layers=cfg.model.num_layers, 
    num_decoder_layers=cfg.model.num_layers, 
    k_neighbors=cfg.model.num_edges, 
    augment_eps=cfg.inference.noise
)

model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model.eval()
model = model.to(dev)
pad = (0, 2, 0, 2) # Pad for 'X' and '-' tokens

# Freeze parameters
for param in model.parameters():
    param.requires_grad = False
    
print("Model loaded successfully.")

Loading model checkpoint...
Model loaded successfully.


In [14]:
def str_split(string, tok):
    if not string:
        return []
    return string.split(tok)

# Read list of PDBs to process and derive chain mapping
with open(cfg.input_list, 'r') as f:
    pdb_lines = f.readlines()
pdb_count = defaultdict(int)
for pdb in pdb_lines:
    pdb_name = pdb.strip().split('|')[0]
    pdb_count[pdb_name] += 1
pdb_list = []
chain_dict = {}
chain_suffixes = []
for pdb in pdb_lines:
    pdb = pdb.strip()
    pdb_info = pdb.split('|')
    if pdb_count[pdb_info[0]] > 1:
        assert len(pdb_info) > 1 # If duplicate pdb entries in list, there must be other identifying info provided
        chain_suffixes.append('|' + '|'.join(pdb_info[1:]))
    else:
        chain_suffixes.append('')
    pdb_list.append(pdb_info[0]) # pdb name
    if len(pdb_info) > 1: # Get designed and fixed chain info
        assert len(pdb_info) == 3
        chain_dict[pdb_info[0] + chain_suffixes[-1]] = [str_split(pdb_info[1], ':'), str_split(pdb_info[2], ':')]
    else: # Set chain_dict to empty list, which means all chains are designed
        chain_dict[pdb_info[0] + chain_suffixes[-1]] = [[], []]
if cfg.chain_dict_json is not None:
    with open(cfg.inference.chain_dict_json, 'r') as f:
        chain_dict = json.load(f)

# Load various configuration dictionaries
fixed_positions_dict, pssm_dict, omit_AA_dict, bias_AA_dict, tied_positions_dict, bias_by_res_dict, omit_AAs_np = process_configs(cfg)
constant = torch.tensor(omit_AAs_np, device=dev)
# Setup Alphabet and Bias
alphabet = 'ACDEFGHIKLMNPQRSTVWYX-'
if cfg.model.vocab == 21:
    alphabet = alphabet[:-1]  # remove gap character

bias_AAs_np = np.zeros(len(alphabet))
if bias_AA_dict:
    for n, AA in enumerate(alphabet):
        if AA in list(bias_AA_dict.keys()):
            bias_AAs_np[n] = bias_AA_dict[AA]
constant_bias = torch.tensor(bias_AAs_np, device=dev)

print(f"Prepared to process {len(pdb_list)} PDBs.")

Prepared to process 6 PDBs.


In [15]:
# Ensure output directories exist for sequences and metrics
os.makedirs(cfg.out_dir, exist_ok=True)

# Filepaths for outputs
filename = os.path.join(cfg.out_dir, cfg.out_name + '.fasta')
decoding_order_filename = os.path.join(cfg.out_dir, cfg.out_name + '_decoding_order.json')
av_loss_filename = os.path.join(cfg.out_dir, cfg.out_name + '_av_loss.csv')
if cfg.inference.write_pdb:
    pdb_out_dir = os.path.join(cfg.out_dir, cfg.out_name + '_pdbs')

# Optimization check logic
skip_calc = False

if cfg.inference.optimization_mode:
    optimized_filename = os.path.join(cfg.out_dir, cfg.out_name + f'_optimized_{cfg.inference.optimization_mode}.fasta')
    
    if cfg.inference.optimize_fasta:
        assert os.path.exists(cfg.inference.optimize_fasta), f"Tried to optimize sequences in {cfg.inference.optimize_fasta}, but the file does not exist."
        print(f"Found existing sequences at {filename}. Loading for optimization...")
        with open(filename, 'r') as f:
            seqs_raw = f.readlines()
        # Parse .fasta
        existing_seqs = {pdb.strip('>').strip(): seq.strip() for pdb, seq in zip(seqs_raw[::2], seqs_raw[1::2])}
        print(f'Saving optimized sequences to filename {optimized_filename}.')
        skip_calc = True
    elif cfg.inference.optimize_pdb:
        print(f"Optimizing existing sequences in pdb files in {cfg.input_dir}. Loading for optimization...")
        existing_seqs = {}
        for pdb, chain_info in zip(pdb_list, chain_suffixes):
            wt_info = parse_PDB_seq_only(os.path.join(cfg.input_dir, pdb + '.pdb'), skip_gaps=cfg.inference.skip_gaps) # Parse .pdb files
            if chain_info:
                _, hidden_chains, vis_chains = chain_info.split('|')
                chain_order = hidden_chains.split(':') + vis_chains.split(':')
                wt_seq = ""
                for chain in chain_order:
                    if chain: wt_seq += wt_info[f'seq_chain_{chain}']
                existing_seqs[pdb + chain_info] = wt_seq
            else:
                existing_seqs[pdb ] = wt_info['seq']
        print(f'Saving optimized sequences to filename {optimized_filename}.')
        skip_calc = True
    else:
        print(f'Saving sequences to filename {filename} and saving optimized sequences to filename {optimized_filename}.')

    # Set up binding energy optimization if requested
    if cfg.inference.binding_energy_optimization in ["both", "only"]:
        assert cfg.inference.binding_energy_json is not None, "Chain separation information required for binding energy optimization"
        with open(cfg.inference.binding_energy_json, 'r') as f:
            binding_energy_chains = json.load(f)
        for pdb, chain_info in zip(pdb_list, chain_suffixes):
            assert pdb + chain_info in binding_energy_chains, f"Chain separation information required for {pdb + chain_info} binding energy optimization"

else:
    print(f'Saving to filename {filename}.')

Saving sequences to filename outputs/example_sequence_test/test_debug.fasta and saving optimized sequences to filename outputs/example_sequence_test/test_debug_optimized_potts.fasta.


In [16]:
# Containers to accumulate outputs and metrics across PDBs
out_seqs = {}
opt_seqs = {}
best_seqs = {}

av_losses = {'pdb': [], 'seq_loss': [], 'nsr': [], 'potts_loss': []}
if cfg.inference.optimize_fasta and os.path.exists(decoding_order_filename):
    with open(decoding_order_filename, 'r') as f:
        decoding_orders = json.load(f)
else:
    decoding_orders = {}

# Iterate over PDBs
for i_pdb, pdb in tqdm(enumerate(pdb_list)):
    input_pdb = os.path.join(cfg.input_dir, pdb + '.pdb')
    pdb_with_chain_suffix = pdb + chain_suffixes[i_pdb]
    # Parse PDB
    pdb_data = parse_PDB(input_pdb, chain_dict[pdb_with_chain_suffix][0] + chain_dict[pdb_with_chain_suffix][1], skip_gaps=cfg.inference.skip_gaps)
    # Featurize
    pdb_chain_dict = {pdb: chain_dict[pdb_with_chain_suffix]}
    X, S_true, mask, _, chain_mask, chain_encoding_all, _, _, _, _, chain_M_pos, omit_AA_mask, residue_idx, _, tied_pos_list_of_lists_list, pssm_coef, pssm_bias, pssm_log_odds, bias_by_res, tied_beta, chain_lens = tied_featurize(
        [pdb_data[0]], dev, pdb_chain_dict, fixed_positions_dict, omit_AA_dict, 
        tied_positions_dict, pssm_dict, bias_by_res_dict, ca_only=False, vocab=cfg.model.vocab
    )
    chain_cuts = [0, *accumulate(chain_lens)]

    pssm_log_odds_mask = (pssm_log_odds > cfg.inference.pssm_threshold).float()
    
    # Set seed if required
    if cfg.inference.fix_decoding_order:
        torch.manual_seed(string_to_int(pdb) + cfg.inference.decoding_order_offset)
    
    # Run Encoder
    h_V, E_idx, h_E, etab = model.run_encoder(X, mask, residue_idx, chain_encoding_all)
    
    # Skip sampling if we are just optimizing existing sequences
    if not skip_calc:
        # etab for Potts energy calculations
        etab_functional = etab_utils.functionalize_etab(etab.clone(), E_idx)
        etab_functional = torch.nn.functional.pad(etab_functional, pad, "constant", 0) # Add padding to account for 'X' and '-' tokens
        sample_records = []
        sample_seq_loss = []
        sample_nsr = []
        sample_nlcpl = []
        
        # Sampling Loop
        for sidx in range(cfg.inference.num_samples):
            if cfg.inference.fix_decoding_order:
                torch.manual_seed(string_to_int(pdb) + cfg.inference.decoding_order_offset + sidx)
            
            randn = torch.randn(chain_mask.shape, device=X.device)

            # Decoder
            if tied_positions_dict is None:
                output_dict, all_probs = model.decoder(
                    h_V, E_idx, h_E, randn, S_true, chain_mask, chain_encoding_all, residue_idx, mask=mask, 
                    temperature=cfg.inference.temperature, omit_AAs_np=omit_AAs_np, bias_AAs_np=bias_AAs_np, 
                    chain_M_pos=chain_M_pos, omit_AA_mask=omit_AA_mask, pssm_coef=pssm_coef, pssm_bias=pssm_bias, 
                    pssm_multi=cfg.inference.pssm_multi, pssm_log_odds_flag=bool(cfg.inference.pssm_log_odds_flag), 
                    pssm_log_odds_mask=pssm_log_odds_mask, pssm_bias_flag=bool(cfg.inference.pssm_bias_flag), 
                    bias_by_res=bias_by_res
                )
            else:
                output_dict, all_probs = model.tied_decoder(
                    h_V, E_idx, h_E, randn, S_true, chain_mask, chain_encoding_all, residue_idx, mask=mask, 
                    temperature=cfg.inference.temperature, omit_AAs_np=omit_AAs_np, bias_AAs_np=bias_AAs_np, 
                    chain_M_pos=chain_M_pos, omit_AA_mask=omit_AA_mask, pssm_coef=pssm_coef, pssm_bias=pssm_bias, 
                    pssm_multi=cfg.inference.pssm_multi, pssm_log_odds_flag=bool(cfg.inference.pssm_log_odds_flag), 
                    pssm_log_odds_mask=pssm_log_odds_mask, pssm_bias_flag=bool(cfg.inference.pssm_bias_flag), 
                    tied_pos=tied_pos_list_of_lists_list[0], tied_beta=tied_beta, bias_by_res=bias_by_res
                )

            # Metrics
            log_probs = torch.log(all_probs)
            mask_for_loss = mask * chain_mask
            _, av_seq_loss, nsr = loss_nll(S_true, torch.nan_to_num(log_probs, nan=0.1, posinf=0.1, neginf=0.1), chain_mask)
            nsr = torch.sum((nsr * mask_for_loss).float()) / torch.sum(mask_for_loss)
            sample_seq_loss.append(av_seq_loss.cpu().item())
            sample_nsr.append(nsr.cpu().item())
            _, av_nlcpl_loss = nlcpl(etab, E_idx, S_true, chain_mask)
            sample_nlcpl.append(av_nlcpl_loss.cpu().item())

            # Convert to String
            seq_str = "".join(etab_utils.ints_to_seq_torch(output_dict['S'][0]))

            # Potts Energy
            seq_tensor = output_dict['S'][0].unsqueeze(0).to(dtype=torch.int64, device=E_idx.device)
            total_energy, _, _ = etab_utils.calc_eners(etab_functional, E_idx, seq_tensor.unsqueeze(1), None)
            total_energy = total_energy.squeeze().cpu().item()
            
            sample_records.append({
                'sample_idx': sidx, 'seq': seq_str, 'energy': total_energy, 
                'decoding_order': output_dict['decoding_order']
            })

        # Sort and Store
        sample_records = sorted(sample_records, key=lambda x: x['energy'])
        for k, rec in enumerate(sample_records):
            sidx = rec['sample_idx']
            sample_suffix = f"_{sidx}" if cfg.inference.num_samples != 1 else ''
            out_seqs[pdb_with_chain_suffix + sample_suffix] = ':'.join(rec['seq'][a:b] for a, b in zip(chain_cuts, chain_cuts[1:]))

            if pdb not in decoding_orders:
                decoding_orders[pdb_with_chain_suffix] = {}
            if cfg.inference.num_samples == 1:
                decoding_orders[pdb_with_chain_suffix] = rec['decoding_order'].squeeze().cpu().numpy().tolist()
            else:
                decoding_orders[pdb_with_chain_suffix][sample_suffix.split('_')[1]] = rec['decoding_order'].squeeze().cpu().numpy().tolist()

            av_losses['pdb'].append(pdb_with_chain_suffix + sample_suffix)
            av_losses['seq_loss'].append(sample_seq_loss[sidx])
            av_losses['nsr'].append(sample_nsr[sidx])
            av_losses['potts_loss'].append(sample_nlcpl[sidx])

            if k == 0: # Save best sequence and sample number
                best_seqs[pdb_with_chain_suffix] = (out_seqs[pdb_with_chain_suffix + sample_suffix], sidx)

    # Optimization Step (Optional)
    if cfg.inference.optimization_mode:
        # Setup for binding energy optimization
        if cfg.inference.binding_energy_optimization in ["both", "only"]:
            partitions = binding_energy_chains[pdb_with_chain_suffix]
            # Ensure chains are ordered correctly in partitions
            chain_order_indices = {chain: i_chain for i_chain, chain in enumerate(pdb_data[0]['chain_order'])}
            index_order_chains = {i_chain: chain for i_chain, chain in enumerate(pdb_data[0]['chain_order'])}
            for i_partition, partition in enumerate(partitions):
                partition_indices = sorted([chain_order_indices[chain] for chain in partition])
                partitions[i_partition] = [index_order_chains[chain_index] for chain_index in partition_indices]
                
            # Get energy tables for separated chains
            partition_etabs = {}
            for i_p, partition in enumerate(partitions):
                partition_etabs[i_p] = get_etab(model, pdb_data, cfg, partition)

            # Define mapping of residues to partition and partition interface mask
            partition_index = chain_to_partition_map(chain_encoding_all, pdb_data[0]['chain_order'], partitions)
            inter_mask = inter_partition_contact_mask(X[:,:,1], partition_index, cfg.inference.binding_energy_cutoff)
        else:
            partition_etabs, partition_index, inter_mask = None, None, None
        
        # Re-encode for optimization context
        h_EX_encoder = cat_neighbors_nodes(torch.zeros_like(h_V), h_E, E_idx)
        h_EXV_encoder = cat_neighbors_nodes(h_V, h_EX_encoder, E_idx)
        
        # Optimize sequences associated with this PDB
        source_seqs = existing_seqs if skip_calc else out_seqs
        
        current_pdb_keys = [k for k in source_seqs.keys() if k.startswith(pdb_with_chain_suffix)]
        
        for key in current_pdb_keys:
            seq_to_opt = source_seqs[key].replace(':', '')
            suffix_key = key[len(pdb_with_chain_suffix):] if len(key) > len(pdb_with_chain_suffix) else ''
            
            stored_decoding = None
            if pdb in decoding_orders:
                if cfg.inference.num_samples == 1:
                    decoding_order = decoding_orders[pdb_with_chain_suffix]
                else:
                    decoding_order = decoding_orders[pdb_with_chain_suffix].get(suffix_key, None)
            else:
                if cfg.inference.fix_decoding_order:
                    if cfg.inference.num_samples != 1:
                        suffix_add = int(suffix_key.split('_')[1])
                    else:
                        suffix_add = 0
                    torch.manual_seed(string_to_int(pdb) + cfg.inference.decoding_order_offset + suffix_add)
                randn = torch.randn(chain_mask.shape, device=X.device)
                decoding_order = torch.argsort((chain_mask+0.0001)*(torch.abs(randn))).squeeze().cpu().numpy().tolist()
                if cfg.inference.num_samples == 1:
                    decoding_orders[pdb_with_chain_suffix] = decoding_order
                else:
                    decoding_orders[pdb_with_chain_suffix][suffix_key.split('_')[1]] = decoding_order
                
            opt_seq = optimize_sequence(
                seq_to_opt, etab, E_idx, mask*chain_M_pos, chain_mask, cfg.inference.optimization_mode, 
                etab_utils.seq_to_ints, cfg.inference.optimization_temperature, constant, constant_bias, 
                bias_by_res, cfg.inference.pssm_bias_flag, pssm_coef, pssm_bias, cfg.inference.pssm_multi,
                cfg.inference.pssm_log_odds_flag, pssm_log_odds_mask, omit_AA_mask, model, h_E, h_EXV_encoder, h_V, 
                decoding_order=decoding_order, partition_etabs=partition_etabs, partition_index=partition_index,
                inter_mask=inter_mask, binding_optimization=cfg.inference.binding_energy_optimization, vocab=cfg.model.vocab
            )
            opt_seq = etab_utils.ints_to_seq_torch(opt_seq)
            opt_seq = ':'.join(opt_seq[a:b] for a, b in zip(chain_cuts, chain_cuts[1:]))
            opt_seqs[key] = opt_seq

            if cfg.inference.num_samples == 1 or (not skip_calc and int(suffix_key.split('_')[1]) == best_seqs[pdb_with_chain_suffix][1]): # Overwrite best sequence if on appropriate sample
                if pdb_with_chain_suffix in best_seqs:
                    suffix = best_seqs[pdb_with_chain_suffix][1]
                else:
                    suffix = 0
                best_seqs[pdb_with_chain_suffix] = (opt_seq, suffix)

print("Processing complete.")

6it [00:07,  1.29s/it]

Processing complete.


In [18]:
# Write Optimized Sequences (if any)
if cfg.inference.optimization_mode:
    print(f"Writing optimized sequences to {optimized_filename}...")
    with open(optimized_filename, 'w') as f:
        for pdb_name, seq in opt_seqs.items():
            f.write('>' + pdb_name + '\n' + seq + '\n')

# Write Decoding Orders
if cfg.inference.optimization_mode or cfg.inference.fix_decoding_order:
    if not os.path.exists(decoding_order_filename) or not cfg.inference.optimize_fasta:
        print(f"Writing decoding orders to {decoding_order_filename}...")
        with open(decoding_order_filename, 'w') as f:
            json.dump(decoding_orders, f)

# Write Sampled Sequences and Metrics (only if sampling occured)
if not skip_calc:
    print(f"Writing sampled sequences to {filename}...")
    with open(filename, 'w') as f:
        for pdb_name, seq in out_seqs.items():
            f.write('>' + pdb_name + '\n' + seq + '\n')
            
    print(f"Writing metrics to {av_loss_filename}...")
    av_losses_df = pd.DataFrame(av_losses)
    av_losses_df.to_csv(av_loss_filename, index=None)

# Write new .pdb files (if requested)
if cfg.inference.write_pdb:
    print(f"Writing new .pdb files to {pdb_out_dir}")
    rewrite_pdb_sequences(best_seqs, cfg.input_dir, pdb_out_dir)

print("All outputs saved.")

Writing optimized sequences to outputs/example_sequence_test/test_debug_optimized_potts.fasta...
Writing decoding orders to outputs/example_sequence_test/test_debug_decoding_order.json...
Writing sampled sequences to outputs/example_sequence_test/test_debug.fasta...
Writing metrics to outputs/example_sequence_test/test_debug_av_loss.csv...
Writing new .pdb files to outputs/example_sequence_test/test_debug_pdbs
All outputs saved.
